# Recency, frequency, monetary (RFM) segmentation

* Recency - mede o quão recente foi a última compra de cada cliente
* Frequency - mede quantas compras o cliente fez nos últimos 12 meses
* MonetaryValue - mede quanto o cliente gastou nos últimos 12 meses. Usaremos esses valores para atribuir clientes a segmentos RFM.

## Read Data

In [18]:
import pandas as pd
import numpy as np
import datetime

In [6]:
d = {'CustomerID': [0,1,2,3,4,5,6,7], 
     'Spend': [137, 335,172,335,303,233,244,229], 
     'Recency_Days': [37,235,396,72,255,393,203,133]}
data = pd.DataFrame(d)

In [7]:
spend_quartiles = pd.qcut(data['Spend'],q=4,labels=range(1,5))
data['Spend_Quartile'] = spend_quartiles
data.sort_values('Spend')

,CustomerID,Spend,Recency_Days,Spend_Quartile
0,0,137,37,1
2,2,172,396,1
7,7,229,133,2
5,5,233,393,2
6,6,244,203,3
4,4,303,255,3
1,1,335,235,4
3,3,335,72,4


In [9]:
# Store labels from 4 to 1 in a decreasing order
r_labels = list(range(4, 0, -1))
recency_quartiles = pd.qcut(data['Recency_Days'], q=4, labels=r_labels)
data['Recency_Quartile'] = recency_quartiles 

print(data.sort_values('Recency_Days'))

   CustomerID  Spend  Recency_Days Spend_Quartile Recency_Quartile
0           0    137            37              1                4
3           3    335            72              4                4
7           7    229           133              2                3
6           6    244           203              3                3
1           1    335           235              4                2
4           4    303           255              3                2
5           5    233           393              2                1
2           2    172           396              1                1


## Calculating RFM metrics

In [60]:
online = pd.read_csv('online.csv',';')
online['InvoiceDate'] = pd.to_datetime(online['InvoiceDate'])
online['TotalSum'] = online['Quantity'] * online['UnitPrice']
online.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSum
0,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25 08:26:00,2.10,14286,United Kingdom,12.60
1,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-20 11:56:00,1.45,16360,United Kingdom,1.45
2,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14 13:35:00,3.75,13933,United Kingdom,22.50
3,578307,72349B,SET/6 PURPLE BUTTERFLY T-LIGHTS,1,2011-11-23 15:53:00,2.10,17290,United Kingdom,2.10
4,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25 13:36:00,5.95,17663,United Kingdom,17.85


In [61]:
snapshot_date = datetime.datetime(2011,12, 10)
datamart = online.groupby(['CustomerID']).agg({
    'InvoiceDate':lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'count',
    'TotalSum':'sum'})

In [62]:
# Rename the columns 
datamart.rename(columns={'InvoiceDate': 'Recency',
                         'InvoiceNo': 'Frequency',
                         'TotalSum': 'MonetaryValue'}, inplace=True)

In [63]:
datamart.head()

,Recency,Frequency,MonetaryValue
CustomerID,,,
12747,22,27,992.82
12748,4,967,7522.06
12749,22,37,813.45
12820,44,17,268.02
12822,70,9,146.15


## Building RFM segments

In [64]:
#Recency quartile

r_labels=range(4,0,-1)
r_quartiles = pd.qcut(datamart['Recency'],4,labels=r_labels)
datamart = datamart.assign(R=r_quartiles)

In [65]:
datamart.head()

,Recency,Frequency,MonetaryValue,R
CustomerID,,,,
12747,22,27,992.82,4
12748,4,967,7522.06,4
12749,22,37,813.45,4
12820,44,17,268.02,3
12822,70,9,146.15,2


In [66]:
# Frequency and Monetary quartiles

f_labels = range(1,5)
m_labels = range(1,5)

f_quartiles = pd.qcut(datamart['Frequency'],4,labels=f_labels)
m_quartiles = pd.qcut(datamart['MonetaryValue'],4,labels=m_labels)

datamart = datamart.assign(F=f_quartiles.values)
datamart = datamart.assign(M=m_quartiles)

datamart.head()

,Recency,Frequency,MonetaryValue,R,F,M
CustomerID,,,,,,
12747,22,27,992.82,4,4,4
12748,4,967,7522.06,4,4,4
12749,22,37,813.45,4,4,4
12820,44,17,268.02,3,3,3
12822,70,9,146.15,2,2,3


In [67]:
# RFM Segment
def join_rfm(x):
    return str(x['R']) + str(x['F']) + str(x['M'])

In [68]:
datamart['RFM_segment'] = datamart.apply(join_rfm,axis=1)
datamart['RFM_score'] = datamart[['R','F','M']].sum(axis=1)

In [69]:
datamart

,Recency,Frequency,MonetaryValue,R,F,M,RFM_segment,RFM_score
CustomerID,,,,,,,,
12747,22,27,992.82,4,4,4,444,12.0
12748,4,967,7522.06,4,4,4,444,12.0
12749,22,37,813.45,4,4,4,444,12.0
12820,44,17,268.02,3,3,3,333,9.0
12822,70,9,146.15,2,2,3,223,7.0
...,...,...,...,...,...,...,...,...
18280,159,2,38.70,2,1,1,211,4.0
18281,3,2,31.80,4,1,1,411,6.0
18282,300,2,30.70,1,1,1,111,3.0


### Analyzing RFM table

In [70]:
datamart.groupby('RFM_segment').size().sort_values(ascending=False)[:10]

RFM_segment
444    347
111    325
344    197
211    184
433    136
222    126
122    125
311    125
333    117
233    108
dtype: int64

Vamos primeiro revisar os 10 maiores segmentos de RFM. Como podemos ver, os segmentos de RFM com classificação mais baixa e mais alta estão entre os maiores. É sempre a melhor prática investigar o tamanho dos segmentos antes de usá-los para direcionamento ou outros aplicativos de negócios.

In [71]:
datamart[datamart['RFM_segment']=='111'][:5]

,Recency,Frequency,MonetaryValue,R,F,M,RFM_segment,RFM_score
CustomerID,,,,,,,,
12837,173,2,10.55,1,1,1,111,3.0
12852,294,2,32.55,1,1,1,111,3.0
12902,264,4,42.03,1,1,1,111,3.0
12922,336,4,57.24,1,1,1,111,3.0
12929,341,3,42.90,1,1,1,111,3.0


Outro aspecto prático dessa segmentação é que ela permite fazer uma seleção simples de clientes com base em seu segmento de RFM. Neste caso, selecionamos o segmento RFM inferior com a segmentação mais baixa de 111.

In [72]:
dt1 = datamart.groupby('RFM_score').agg({'Recency':'mean',
                                  'Frequency':'mean',
                                  'MonetaryValue':['mean','count']}).round(1)
dt1

Recency Frequency MonetaryValue      
             mean      mean          mean count
RFM_score                                      
3.0         280.0       2.0          28.3   325
4.0         199.9       3.2          46.2   359
5.0         163.2       4.5          77.1   428
6.0         119.3       6.3         115.5   432
7.0         104.9       9.5         179.3   378
8.0          69.7      12.4         234.7   361
9.0          58.0      16.7         323.5   358
10.0         40.6      25.9         401.4   368
11.0         26.7      39.4        1093.6   345
12.0         10.1      78.5        1459.2   347

In [78]:
def segment_me(df):
    if df['RFM_score'] >= 10:
        return 'Gold'
    elif (df['RFM_score']>=6 and (df['RFM_score']<10)):
        return 'Silver'
    else:
        return 'Bronze'

In [80]:
datamart['General_Segment'] = datamart.apply(segment_me, axis=1)
datamart.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFM_segment,RFM_score,General_Segment
CustomerID,,,,,,,,,
12747,22,27,992.82,4,4,4,444,12.0,Gold
12748,4,967,7522.06,4,4,4,444,12.0,Gold
12749,22,37,813.45,4,4,4,444,12.0,Gold
12820,44,17,268.02,3,3,3,333,9.0,Silver
12822,70,9,146.15,2,2,3,223,7.0,Silver


In [81]:
dt2 = datamart.groupby('General_Segment').agg({'Recency':'mean',
                                        'Frequency':'mean',
                                        'MonetaryValue':['mean','count']}).round(1)
dt2

Recency Frequency MonetaryValue      
                   mean      mean          mean count
General_Segment                                      
Bronze            209.2       3.4          52.9  1112
Gold               26.1      47.5         973.0  1060
Silver             89.7      10.9         208.1  1529